In [5]:
import pandas as pd
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
import os
import azure.ai.ml._artifacts._artifact_utilities as artifact_utils


# Connexion à Azure ML
ml_client = MLClient.from_config(credential=DefaultAzureCredential())

# Récupération du data asset
data_asset = ml_client.data.get("cinema_data_set", version="3")

ASSET_NAME = "cinema_data_set"
ASSET_VERSION = "3"
LOCAL_PATH = "./downloaded_data"

artifact_utils.download_artifact_from_aml_uri(uri = data_asset.path, destination = "./mlasset/", datastore_operation=ml_client.datastores)

Found the config file in: ./config.json
Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


'./mlasset/'

In [4]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

df_cleaned = df.drop(columns=["Cabin"], errors="ignore")

# Sauvegarde locale du nouveau CSV
cleaned_path = "titanic_data_cleaned.csv"
df_cleaned.to_csv(cleaned_path, index=False)

# Création de la nouvelle version du Data Asset (v2)
new_data_asset = Data(
    path=cleaned_path,
    type=AssetTypes.URI_FILE,
    name="titanic_data_set",  # même nom
    version="2",              # nouvelle version
    description="Version nettoyée sans la colonne Cabin"
)

ml_client.data.create_or_update(new_data_asset)

Uploading titanic_data_cleaned.csv (< 1 MB): 100%|██████████| 61.8k/61.8k [00:00<00:00, 4.85MB/s]




Data({'path': 'azureml://subscriptions/72eb7803-e874-44cb-b6d9-33f2fa3eb88c/resourcegroups/jvangansbergrg/workspaces/myws/datastores/workspaceblobstore/paths/LocalUpload/336f33fafd464f8997bde01c3474f660/titanic_data_cleaned.csv', 'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'uri_file', 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'titanic_data_set', 'description': 'Version nettoyée sans la colonne Cabin', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/72eb7803-e874-44cb-b6d9-33f2fa3eb88c/resourceGroups/jvangansbergrg/providers/Microsoft.MachineLearningServices/workspaces/myws/data/titanic_data_set/versions/2', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/testvmsimplon/code/Users/jvangansberg.ext/cinema-admission', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7f41d47a47f0>, 'serialize': <msre

In [5]:
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# Connexion + chargement du dataset v2
ml_client = MLClient.from_config(credential=DefaultAzureCredential())
data_asset = ml_client.data.get(name="titanic_data_set", version="2")
df = pd.read_csv(data_asset.path)

# Préparation des données (exemple simple)
df = df.dropna()
X = df.drop(columns=["Survived"])
y = df["Survived"]

# Dummy encoding si besoin (ex: pour les colonnes catégorielles)
X = pd.get_dummies(X)

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Démarrer un run MLflow
with mlflow.start_run():
    model = LogisticRegression(max_iter=200)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]

    # Métriques
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_proba)

    # Log avec MLflow
    mlflow.log_param("model_type", "LogisticRegression")
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("roc_auc", roc_auc)

    mlflow.sklearn.log_model(model, "model")

    print(f"✅ Modèle loggé avec MLflow - F1: {f1:.3f}, Precision: {precision:.3f}, Recall: {recall:.3f}, AUC: {roc_auc:.3f}")


Found the config file in: /config.json
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/azureml/mlflow/_protos/aml_service_pb2.py:10: UserWarning: google.protobuf.service module is deprecated. RPC implementations should provide code generator plugins which generate code specific to the RPC implementation. service.py will be removed in Jan 2025
  from google.protobuf import service as _service
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
2025/04/02 10:00:52 WARNING mlflow.models.model: Model

✅ Modèle loggé avec MLflow - F1: 0.759, Precision: 0.830, Recall: 0.698, AUC: 0.858


In [4]:
!conda list --explicit > environment.txt